# Everruns Agent API Example

This notebook demonstrates how to use the Everruns API to:
1. Create an agent with a system prompt
2. Create a session (conversation)
3. Send a message to trigger the agentic loop
4. Read events until the assistant response appears
5. Continue the conversation

## Prerequisites

- Everruns API server running at `http://localhost:9000`
- Python packages: `requests`

In [ ]:
# Install required packages
!pip install requests

In [ ]:
import requests
import json
import time

# Configuration
BASE_URL = "http://localhost:9000"
API_V1 = f"{BASE_URL}/v1"

# Track the last event offset (UUID7) for efficient continuation
last_offset = None  # UUID string, None means start from beginning

def print_json(data):
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2, default=str))

## 1. Health Check

Verify the API server is running.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
response.raise_for_status()
print("API Status:", response.json())

## 2. Create an Agent

An agent is a configuration for an agentic loop with a system prompt.

In [ ]:
agent_data = {
    "name": "Python Example Agent",
    "description": "A helpful assistant created via the Python API",
    "system_prompt": "You are a helpful assistant. Be concise and friendly.",
    "tags": ["example", "python"]
}

response = requests.post(f"{API_V1}/agents", json=agent_data)
response.raise_for_status()
agent = response.json()

agent_id = agent["id"]
print(f"Created agent: {agent_id}")

## 3. Create a Session

A session is an instance of conversation with the agent.

In [ ]:
session_data = {
    "title": "Python API Test Session"
}

response = requests.post(f"{API_V1}/agents/{agent_id}/sessions", json=session_data)
response.raise_for_status()
session = response.json()

session_id = session["id"]
print(f"Created session: {session_id}")

## 4. Send a Message

Sending a user message triggers the agentic loop workflow.

In [ ]:
message_data = {
    "message": {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is 2 + 2? Please explain briefly."}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print(f"Sent message: {response.json()['id']}")

## 5. Read Events

Poll the events endpoint to get the assistant's response. Events include messages (`message.assistant`, `message.tool_call`, `message.tool_result`) and loop lifecycle events.

In [ ]:
def read_until_output(agent_id: str, session_id: str, offset: str = None, timeout: int = 60):
    """Poll events until an assistant message appears.
    
    Uses UUID7-based pagination for efficient event fetching:
    - Pass the last known offset (UUID7) to resume from that point
    - Returns (response_text, next_offset) for continuation
    
    UUID7 is time-ordered, making it ideal for offset-based resumption.
    """
    url = f"{API_V1}/agents/{agent_id}/sessions/{session_id}/events"
    current_offset = offset
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        # Use offset parameter to only fetch new events
        params = {"limit": 100}
        if current_offset:
            params["offset"] = current_offset
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        result = response.json()
        
        events = result.get("data", [])
        next_offset = result.get("next_offset")  # UUID7 string
        has_more = result.get("has_more", False)
        
        for event in events:
            event_type = event["event_type"]
            data = event["data"]
            event_id = event["id"]  # UUID7
            
            # Print event details
            if event_type == "message.agent":
                content = data.get("content", [])
                text_parts = [p["text"] for p in content if p.get("type") == "text"]
                text = "\n".join(text_parts)
                print(f"[{event_type}] {text[:100]}{'...' if len(text) > 100 else ''}")
                # Return the response and the offset (UUID7) to continue from
                return text, next_offset if next_offset else event_id
            elif event_type == "message.tool_call":
                content = data.get("content", [])
                for part in content:
                    if part.get("type") == "tool_call":
                        print(f"[{event_type}] {part.get('name')}({part.get('arguments')})")
            elif event_type == "message.tool_result":
                content = data.get("content", [])
                for part in content:
                    if part.get("type") == "tool_result":
                        result_data = part.get("result", "")
                        print(f"[{event_type}] {str(result_data)[:80]}...")
            elif event_type == "checkpoint":
                # Checkpoint events mark safe resumption points
                status = data.get("status", "")
                last_id = data.get("last_event_id", "")
                print(f"[checkpoint] status={status}, last_id={last_id[:8]}...")
            elif event_type == "LoopCompleted":
                print(f"[{event_type}] iterations={data.get('total_iterations')}")
            elif event_type in ("LoopStarted", "IterationStarted", "LlmCallStarted", "LlmCallCompleted"):
                print(f"[{event_type}]")
            elif event_type == "LoopError":
                print(f"[{event_type}] {data.get('error')}")
                return None, next_offset if next_offset else current_offset
        
        # Update offset for next poll
        if next_offset:
            current_offset = next_offset
        
        time.sleep(0.5)
    
    return None, current_offset

# Read events until we get the assistant's response
print("Waiting for assistant response...\n")
assistant_response, last_offset = read_until_output(agent_id, session_id, offset=None)

if assistant_response:
    print(f"\n--- ASSISTANT ---")
    print(assistant_response)
    print(f"\n(Saved offset: {last_offset[:8] if last_offset else None}... for continuation)")
else:
    print("No response received")

## 6. Continue the Conversation

Send another message and read the response.

In [ ]:
# Send follow-up message
message_data = {
    "message": {
        "content": [
            {"type": "text", "text": "What about 3 + 3?"}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print(f"Sent follow-up message (resuming from offset {last_offset[:8] if last_offset else None}...)\n")

# Read response using the saved offset (UUID7) - only fetches NEW events
assistant_response, last_offset = read_until_output(agent_id, session_id, offset=last_offset)

if assistant_response:
    print(f"\n--- ASSISTANT ---")
    print(assistant_response)
    print(f"\n(Saved offset: {last_offset[:8] if last_offset else None}... for next continuation)")

## 7. Cleanup

Delete the session and archive the agent.

In [ ]:
# Delete session
requests.delete(f"{API_V1}/agents/{agent_id}/sessions/{session_id}")
print(f"Deleted session: {session_id}")

# Archive agent
requests.delete(f"{API_V1}/agents/{agent_id}")
print(f"Archived agent: {agent_id}")

## Summary

Core API workflow:

1. `POST /v1/agents` - Create agent
2. `POST /v1/agents/{id}/sessions` - Create session
3. `POST /v1/agents/{id}/sessions/{id}/messages` - Send user message (triggers agentic loop)
4. `GET /v1/agents/{id}/sessions/{id}/events?offset=UUID&limit=M` - Poll events with UUID7-based pagination

### UUID7-Based Event Fetching (Durable Streams Pattern)

The events endpoint uses UUID7 for offset-based pagination:

- `?offset=UUID7` - Fetch events with id > UUID (UUID7 is time-ordered)
- `?limit=M` - Limit number of events per request (default: 100, max: 1000)
- Response includes `next_offset` (UUID7) and `has_more` for continuation

**Why UUID7?**
- Time-ordered: First 48 bits are Unix timestamp in milliseconds
- Already stored as event ID - no separate sequence counter needed
- Globally unique across sessions
- Standard UUID format works with any client

**Benefits:**
- **Efficient**: Only fetch new events, not the entire history
- **Resumable**: Save `next_offset` and resume after disconnection
- **Cacheable**: Historical pages have `Cache-Control: immutable`

### Event Types

Events contain all session activity including messages:

- `message.user` - User message
- `message.agent` - Agent/assistant response with content
- `message.tool_call` - Tool invocation by agent
- `message.tool_result` - Result of tool execution
- `checkpoint` - Safe resumption point (status + last_event_id)
- `LoopStarted`, `LoopCompleted` - Agentic loop lifecycle
- `LlmCallStarted`, `LlmCallCompleted` - LLM call lifecycle

API docs: http://localhost:9000/swagger-ui/